In [1]:
# Imports

import pandas as pd
import numpy as np
import os

In [ ]:
# Define the folder path
data_path = "data/"

# Load datasets
#osebx = pd.read_csv(os.path.join(data_path, "osebx.csv"))
zero_coupon = pd.read_csv(os.path.join(data_path, "zero_coupon.csv"))
exchange_rates = pd.read_csv(os.path.join(data_path, "exchange_rates.csv"))
inflation = pd.read_csv(os.path.join(data_path, "kpi.csv"))